In [1]:
import numpy as np
import mahotas
import cv2
import os
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


E:\condapy\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import itertools

indexes = list(itertools.product([0,1,2,3,4,5,6,7,8,9,10,11],[0,1,2,3,4],[0,1,2,3,4]))
print(indexes)

[(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 0, 3), (0, 0, 4), (0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 2, 0), (0, 2, 1), (0, 2, 2), (0, 2, 3), (0, 2, 4), (0, 3, 0), (0, 3, 1), (0, 3, 2), (0, 3, 3), (0, 3, 4), (0, 4, 0), (0, 4, 1), (0, 4, 2), (0, 4, 3), (0, 4, 4), (1, 0, 0), (1, 0, 1), (1, 0, 2), (1, 0, 3), (1, 0, 4), (1, 1, 0), (1, 1, 1), (1, 1, 2), (1, 1, 3), (1, 1, 4), (1, 2, 0), (1, 2, 1), (1, 2, 2), (1, 2, 3), (1, 2, 4), (1, 3, 0), (1, 3, 1), (1, 3, 2), (1, 3, 3), (1, 3, 4), (1, 4, 0), (1, 4, 1), (1, 4, 2), (1, 4, 3), (1, 4, 4), (2, 0, 0), (2, 0, 1), (2, 0, 2), (2, 0, 3), (2, 0, 4), (2, 1, 0), (2, 1, 1), (2, 1, 2), (2, 1, 3), (2, 1, 4), (2, 2, 0), (2, 2, 1), (2, 2, 2), (2, 2, 3), (2, 2, 4), (2, 3, 0), (2, 3, 1), (2, 3, 2), (2, 3, 3), (2, 3, 4), (2, 4, 0), (2, 4, 1), (2, 4, 2), (2, 4, 3), (2, 4, 4), (3, 0, 0), (3, 0, 1), (3, 0, 2), (3, 0, 3), (3, 0, 4), (3, 1, 0), (3, 1, 1), (3, 1, 2), (3, 1, 3), (3, 1, 4), (3, 2, 0), (3, 2, 1), (3, 2, 2), (3, 2, 3), (3, 2, 4), (3, 3, 0)

In [3]:
def normalize(values, bounds):
    return [bounds['desired']['lower'] + (x - bounds['actual']['lower']) * (bounds['desired']['upper'] - bounds['desired']['lower']) / (bounds['actual']['upper'] - bounds['actual']['lower']) for x in values]

In [7]:
def get_F(h, s, v):
    h_norm = normalize(
        h,
        {'actual': {'lower': 0, 'upper': 179}, 'desired': {'lower': 0, 'upper': 360}}
    )
    s_norm = normalize(
        s,
        {'actual': {'lower': 0, 'upper': 255}, 'desired': {'lower': 0, 'upper': 1}}
    )
    v_norm = normalize(
        v,
        {'actual': {'lower': 0, 'upper': 255}, 'desired': {'lower': 0, 'upper': 1}}
    )
    
    h_norm = np.array(h_norm).flatten()
    s_norm = np.array(s_norm).flatten()
    v_norm = np.array(v_norm).flatten()
    
    h_d = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0, 11:0}
    s_d = {0:0, 1:0, 2:0, 3:0, 4:0}
    v_d = {0:0, 1:0, 2:0, 3:0, 4:0}


    for i in range(len(h_norm)):
        H = h_norm[i]
        ha = None
        if H >= 346 or H <= 15:
            ha = 0
        elif H >= 16 and H <= 45:
            ha = 1
        elif H >= 46 and H <= 75:
            ha = 2
        elif H >= 76 and H <= 105:
            ha = 3
        elif H >= 106 and H <= 135:
            ha = 4
        elif H >= 136 and H <= 165:
            ha = 5
        elif H >= 166 and H <= 196:
            ha = 6
        elif H >= 196 and H <= 225:
            ha = 7
        elif H >= 226 and H <= 255:
            ha = 8
        elif H >= 256 and H <= 285:
            ha = 9
        elif H >= 286 and H <= 315:
            ha = 10
        elif H >= 316 and H <= 345:
            ha = 11
        if ha not in h_d:
            h_d[ha] = 1
        else:
            h_d[ha] += 1

    for i in range(len(s_norm)):
        S = s_norm[i]
        sa = None
        if S >= 0 and S <= 0.2:
            sa = 0
        elif S > 0.2 and S <= 0.4:
            sa = 1
        elif S > 0.4 and S <= 0.6:
            sa = 2
        elif S > 0.6 and S <= 0.8:
            sa = 3
        elif S > 0.8 and S <= 1:
            sa = 4
        if sa not in s_d:
            s_d[sa] = 1
        else:
            s_d[sa] += 1

    for i in range(len(v_norm)):
        V = v_norm[i]
        va = None
        if V >= 0 and V <= 0.2:
            va = 0
        elif V > 0.2 and V <= 0.4:
            va = 1
        elif V > 0.4 and V <= 0.6:
            va = 2
        elif V > 0.6 and V <= 0.8:
            va = 3
        elif V > 0.8 and V <= 1:
            va = 4
        if va not in v_d:
            v_d[va] = 1
        else:
            v_d[va] += 1
            
    F = np.zeros((300,))
    
    for i in range(12):
        for j in range(5):
            for k in range(5):
                F[indexes.index((i, j, k))] = 5*h_d[i] + 3*s_d[j] + 2*v_d[k]
    return F

In [8]:
def save_video(fname, key_frames):
    count = 0

    cur_dir = fname[0:-4]
    print(cur_dir)
    os.mkdir(cur_dir)

    vidcap = cv2.VideoCapture(fname)
    success, img = vidcap.read()
    print('Read a new frame: ', success)

    while success:
        if count in key_frames:
            print(f'{count} in key frame')
            cv2.imwrite(cur_dir + "\\" + str(count) + ".jpg", img)
        success, img = vidcap.read()
        count += 1
    vidcap.release()

def process_video(fname):
    """
        Uses HU AND Zernike moments

    """
    colour_features = []

    vidcap = cv2.VideoCapture(fname)
    success, img = vidcap.read()

    count = 0
    while success:
        img3 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img2 = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        h, s, v = img2[:,:,0], img2[:,:,1], img2[:,:,2]
        F1 = get_F(h, s, v)
        F1_norm = normalize(
            F1,
            {'actual': {'lower': 0, 'upper': 1000000}, 'desired': {'lower': 0, 'upper': 1}}
        )
        thresh = cv2.threshold(img3, 50, 100, cv2.THRESH_BINARY)[1]
        F2 = np.append(mahotas.features.zernike_moments(thresh, 250), cv2.HuMoments(cv2.moments(img3)).flatten())
        #print(F)

        colour_features.append(np.append(F1_norm, F2))
        if(count%25 == 0):
            print(f"{count} images done")
        success, img = vidcap.read()
        count += 1
    
    vidcap.release()
        
    best_i = 0
    best_sc = -1

    for i in range(3,21):
        kmeans = KMeans(n_clusters=i, random_state=0).fit(colour_features)
        sc = silhouette_score(colour_features, kmeans.labels_)
        if(sc > best_sc):
            best_i = i
            best_sc = sc
        #print(i, silhouette_score(colour_features, kmeans.labels_))

    print(f'Best silhouette score was {best_sc} for k value of {best_i}')

    kmeans = KMeans(n_clusters=best_i, random_state=0).fit(colour_features)
    cents = kmeans.cluster_centers_

    print(len(colour_features), len(kmeans.labels_))

    min_dict = {}
    min_frame= {}
    for i in range(len(kmeans.labels_)):
        t = np.linalg.norm(colour_features[i]-cents[kmeans.labels_[i]])
        if kmeans.labels_[i] not in min_dict:
            min_dict[kmeans.labels_[i]] = t
            min_frame[kmeans.labels_[i]] = i
        else:
            if(t < min_dict[kmeans.labels_[i]]):
                min_dict[kmeans.labels_[i]] = t
                min_frame[kmeans.labels_[i]] = i

    print(min_frame)

    key_frames = list(min_frame.values())

    save_video(fname, key_frames)


In [9]:
import os
from os import listdir

import cv2
for root, dirs, files in os.walk("C:\\Users\\Souvik\\Desktop\\OVPP\\76 bin_hu_zernike", topdown=False):
    for name in files:
        print(os.path.join(root, name))
        process_video(os.path.join(root, name))

C:\Users\Souvik\Desktop\OVPP\76 bin_hu_zernike\v15.flv
0 images done
25 images done
50 images done
75 images done
100 images done
125 images done
150 images done
175 images done
200 images done
225 images done
250 images done
275 images done
300 images done
325 images done
350 images done
375 images done
400 images done
425 images done
450 images done
475 images done
500 images done
525 images done
550 images done
575 images done
600 images done
625 images done
650 images done
675 images done
700 images done
725 images done
750 images done
775 images done
800 images done
825 images done
850 images done
875 images done
900 images done
925 images done
950 images done
975 images done
1000 images done
1025 images done
1050 images done
1075 images done
1100 images done
1125 images done
1150 images done
1175 images done
1200 images done
1225 images done
1250 images done
1275 images done
1300 images done
1325 images done
1350 images done
1375 images done
Best silhouette score was 0.7587848795